# Import des outils / jeu de données

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
np.random.seed(0)
sns.set_theme()

In [ ]:
df = pd.read_csv("data/data.csv", sep=",", index_col="ID", parse_dates=True)

In [ ]:
df.head()

## Variables globales

In [ ]:
var_numeriques = [
    "Year_Birth",
    "Income",
    "Recency",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
    "NumDealsPurchases",
    "NumWebPurchases",
    "NumCatalogPurchases",
    "NumStorePurchases",
    "NumWebVisitsMonth",
]

In [ ]:
var_categoriques = [
    "Education",
    "Marital_Status",
    "Kidhome",
    "Teenhome",
    "Complain",
    "AcceptedCmp1",
    "AcceptedCmp2",
    "AcceptedCmp3",
    "AcceptedCmp4",
    "AcceptedCmp5",
    "Response",
]

In [ ]:
df[var_categoriques] = df[var_categoriques].astype(str).astype("category")

# Nettoyage des données

## Données parasites


### Variables

Dans le jeu de données, 2 colonnes sont non documentées et contiennent des valeurs constantes :
- `Z_CostContact`
- `Z_Revenue`

**Variable `Z_CostContact`**

In [ ]:
print(
    f'Il y a {np.sum(df["Z_CostContact"] == 3)} / {len(df)} individus pour lesquelles la variable Z_CostContact vaut 3.'
)

In [ ]:
df["Z_CostContact"].hist()

**Variable `Z_Revenue`**

In [ ]:
print(
    f'Il y a {np.sum(df["Z_Revenue"] == 11)} / {len(df)} individus pour lesquelles la variable Z_Revenue vaut 11.'
)

In [ ]:
df["Z_Revenue"].hist()

Nous décidons de les supprimer, car elles sont parasites et ne fournissent pas d'information.

In [ ]:
df.drop(columns=["Z_CostContact", "Z_Revenue"], inplace=True)

### Valeurs

#### Statut marital

In [ ]:
sns.histplot(y=df["Marital_Status"])

Parmi les situations maritales, on observe "absurde" et "YOLO" (You Only Live Once, _on ne vit qu'une fois_).

Comme on ne peut pas déduire ce que veut dire "absurde" dans ce contexte, on se décide de supprimer l'individu.
De même, on suppose que la personne ayant répondu "YOLO", donc ayant répondu de façon humoristique, doit être supprimée du jeu de données.

In [ ]:
df.drop(index=df.loc[df["Marital_Status"] == "YOLO"].index, inplace=True)
df.drop(index=df.loc[df["Marital_Status"] == "Absurd"].index, inplace=True)

In [ ]:
# On retire les options "YOLO" et "Absurd" de nos variables catégoriques.
df["Marital_Status"] = df["Marital_Status"].cat.remove_categories(["YOLO", "Absurd"])

In [ ]:
sns.histplot(y=df["Marital_Status"])

## Données dupliquées

### Individus

In [ ]:
sum(df.duplicated())

Nous observons qu'il y a 182 lignes qui sont dupliquées.

Nous pouvons supposer que les données ont été fusionnées à partir de plusieurs sources, et que ces sources avaient des individus en commun : nous décidons donc de supprimer les doublons.

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
sum(df.duplicated())

## Variables pas assez informatives

In [ ]:
# todo: dire que la variable "Complain" (si le client s'est plaint ou non), n'apporte pas assez d'information pour être gardée, car il y a moins de 1% des observations qui ont une classe autre que "0", et qu'en plus ce n'est pas central à notre étude

In [ ]:
# todo: essayer quand même une visualisation des individus s'étant plaint par rapport à nos variables cibles, au cas où...

In [ ]:
sns.histplot(df["Complain"], shrink=0.5)

In [ ]:
print(
    f'Il y a {100 * df["Complain"].astype(int).sum() / len(df["Complain"]):.2f}% des individus qui se sont plaints.'
)

## Standardisation

### Modalités de variables catégoriques

In [ ]:
sns.histplot(y=df["Marital_Status"])

Sans plus d'information, "Alone" est une autre formulation de "Single", on décide donc de remplacer "Alone" par "Single".

In [ ]:
df["Marital_Status"].replace("Alone", "Single", inplace=True)

In [ ]:
sns.histplot(y=df["Marital_Status"])

## Valeurs extrêmes

### Income

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 3))
sns.boxplot(df["Income"], width=0.25, ax=ax[0])
sns.histplot(df["Income"], kde=True, ax=ax[1])
plt.show()

In [ ]:
# todo: texte pour commenter

In [ ]:
df["Income"][df["Income"] > 200_000]

Pour nous, il s'agit d'une valeur aberrante à supprimer du jeu de données car "666 666" ne semble pas être une valeur réelle (6 fois le nombre 6...).

In [ ]:
df.drop(index=df[df["Income"] > 200_000].index, inplace=True)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 3))
sns.boxplot(df["Income"], width=0.25, ax=ax[0])
sns.histplot(df["Income"], kde=True, ax=ax[1])
plt.show()

In [ ]:
# todo: est-ce qu'on retire les valeurs au-dessus de 120 000 ? comment on le justifie ?
# => ce serait plutôt des individus ATYPIQUES, à enlever mais garder de côté pour l'analyse a posteriori

In [ ]:
df.drop(index=df[df["Income"] > 120_000].index, inplace=True)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 3))
sns.boxplot(df["Income"], width=0.25, ax=ax[0])
sns.histplot(df["Income"], kde=True, ax=ax[1])
plt.show()

### Year Date

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 3))
sns.boxplot(df["Year_Birth"], width=0.25, ax=ax[0])
sns.histplot(df["Year_Birth"], kde=True, ax=ax[1])
plt.show()

In [ ]:
df[df["Year_Birth"] < 1920]

Comme les personnes les plus âgées au monde ont très rarement plus de 100 ans, et que nous sommes en 2023, on peut sans problème estimer que les trois dates de naissance avant 1920 sont erronées.

Pour les deux années 1893 et 1899, on peut s'imaginer que les personnes ont fait une faute de frappe, et voulaient écrire, respectivement, 1983 et 1989.

In [ ]:
df.loc[df["Year_Birth"] == 1893, "Year_Birth"] = 1983
df.loc[df["Year_Birth"] == 1899, "Year_Birth"] = 1989

Pour l'année 1900, il est difficile d'imaginer une faute de frappe, il s'agit peut-être de la valeur par défaut.
On peut tout simplement supprimer la ligne correspondante.

In [ ]:
df.drop(index=df.loc[df["Year_Birth"] == 1900, "Year_Birth"].index, inplace=True)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 3))
sns.boxplot(df["Year_Birth"], width=0.25, ax=ax[0])
sns.histplot(df["Year_Birth"], kde=True, ax=ax[1])
plt.show()

## Valeurs manquantes

In [ ]:
sns.heatmap(
    df.isna(), cbar=False
)  # todo: mettre les graphiques à côté (horizontalement)

In [ ]:
sns.displot(
    data=df.isna().melt(value_name="missing"),
    y="variable",
    hue="missing",
    multiple="fill",
    aspect=1.25,
)

In [ ]:
plt.title("Nombre de valeurs manquantes pour chaque variable")
df.isna().sum().plot.barh()

In [ ]:
# todo : commenter

In [ ]:
# TODO: supprimer valeurs manquantes OU remplacer par la moyenne ?

In [ ]:
# todo: expliquer POURQUOI ces données sont manquantes (notre hypothèse du moins), et expliquer notre démarche en conséquence

### Remplacement par la moyenne

In [ ]:
df["Income"][df["Income"].isna()]

In [ ]:
df["Income"].fillna(df["Income"].mean(), inplace=True)

In [ ]:
# Vérification du bon remplacement des valeurs manquantes
df["Income"].isna().sum()

# Sauvegarde du Dataframe

In [ ]:
df.to_csv("data/data-cleaned.csv")